In [2]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader

In [3]:

def calculate_macd(df):
    # techinical indicator 
    short_term = 12
    long_term = 26
    signal_period = 9

    df['EMA12'] = df['Close/Last'].ewm(span=short_term,adjust=False).mean()
    df['EMA26'] = df['Close/Last'].ewm(span=long_term,adjust=False).mean()

    df['MACD Line'] = df['EMA12'] - df['EMA26']
    df['Signal Line'] = df['MACD Line'].ewm(span=signal_period,adjust=False).mean()
    df['MACD'] = df['MACD Line'] - df['Signal Line']
    df.drop(['EMA12','EMA26','MACD Line','Signal Line'],inplace=True,axis=1)
    return df

In [4]:
def calucate_atr(data):
    # techinical indicator 
    atr_period = 14
    data['H-L'] = data['High'] - data['Low']
    data['H-PC'] = (data['High'] - data['Close/Last'].shift(1)).abs()
    data['L-PC'] = (data['Low'] - data['Close/Last'].shift(1)).abs()

    data['TR'] = data[['H-L','H-PC','L-PC']].max(axis=1)

    data['ATR'] = data['TR'].rolling(atr_period).mean()
    data.drop(['H-L','H-PC','L-PC','TR'],inplace=True,axis=1)
    return data

In [5]:

def calculate_rsi(data):
    # techinical indicator 
    rsi_period = 14
    data['Price Change'] = data['Close/Last'].diff()
    data['Gain'] = data['Price Change'].apply(lambda x: x if x > 0 else 0)
    data['Loss'] = data['Price Change'].apply(lambda x: -x if x < 0 else 0)
    data['Average Gain'] = data['Gain'].rolling(window = rsi_period).mean()
    data['Average Loss'] = data['Loss'].rolling(window = rsi_period).mean()
    data['Relative Strength'] = data['Average Gain']/data['Average Loss']
    data['RSI'] = 100 - (100/(1+data['Relative Strength']))

    data.drop(['Price Change','Gain','Loss','Average Gain','Average Loss','Relative Strength'],inplace=True,axis=1)

    return data

In [26]:
def get_data(path):
    df = pd.read_csv(path)
    df.Date = df['Date'].apply(lambda x: pd.to_datetime(x,format="%m/%d/%Y"))
    return df
df = get_data("../data/HistoricalData_1698863282356.csv")

In [44]:
def calculate_whole(data_df, path, date_format, feature):
    df_add = pd.read_csv(path)
    print(df_add)
    df_add.Date = df_add['Date'].apply(lambda x: pd.to_datetime(x,format=date_format))
    merged_df = pd.merge_ordered(data_df, df_add, on='Date', how='left')
    return merged_df

In [45]:
def calculate_column(data_df, path, date_format, feature):
    df_add = pd.read_csv(path)
    print(df_add)
    df_add.Date = df_add['Date'].apply(lambda x: pd.to_datetime(x,format=date_format))
    column = "Adj Close"
    merged_df = pd.merge_ordered(data_df, df_add[['Date', column]], on='Date', how='left')
    merged_df.rename(columns={column: feature}, inplace=True)
    return merged_df

In [79]:
def calculate_monthly(data_df, path, date_format, feature):
    df_add = pd.read_csv(path)
    print(df_add)
    df_add.Date = df_add['DATE'].apply(lambda x: pd.to_datetime(x,format=date_format))
    df_add['Date'] = pd.to_datetime(df_add['DATE']).dt.to_period('M').dt.to_timestamp()
    df_add['YearMonth'] = df_add['Date'].dt.to_period('M')
    data_df['YearMonth'] = data_df['Date'].dt.to_period('M')
    print(data_df)
    merged_df = pd.merge_ordered(data_df, df_add, left_on='YearMonth', right_on='YearMonth', how='left')
    merged_df.drop(['YearMonth', 'DATE', 'Date_y'], axis=1, inplace=True)
    merged_df.rename(columns={'Date_x': 'Date'}, inplace=True)
    return merged_df

In [82]:
if __name__ == "__main__":
    df = get_data("../data/HistoricalData_1698863282356.csv")
    df = calculate_macd(df)    # Moving Average Convergence Divergence
    df = calucate_atr(df)      # market volatility
    df = calculate_rsi(df)
    df = calculate_whole(df, "../data/EFFR.csv", date_format = "%d-%m-%Y", feature = "EFFR")
    df = calculate_column(df, "../data/VIX.csv", date_format = "%d-%m-%Y", feature = "VIX" )
    df = calculate_column(df, "../data/DX-Y.NYB.csv", date_format = "%Y-%m-%d", feature = "USDX" )
    df = calculate_monthly(df, "../data/UNRATE.csv", date_format = "%Y-%m-%d", feature = "UNRATE" )
    df = calculate_monthly(df, "../data/UMCSENT.csv", date_format = "%Y-%m-%d", feature = "UMCSENT" )
    df.to_csv("../data/final_dataset.csv",index=False)

            Date  EFFR
0     02-01-2008  4.11
1     03-01-2008  4.25
2     04-01-2008  4.18
3     07-01-2008  4.27
4     08-01-2008  4.27
...          ...   ...
4125  25-10-2023  5.33
4126  26-10-2023  5.33
4127  27-10-2023  5.33
4128  30-10-2023  5.33
4129  31-10-2023  5.33

[4130 rows x 2 columns]
            Date       Open       High        Low      Close  Adj Close  \
0     02-01-1990  17.240000  17.240000  17.240000  17.240000  17.240000   
1     03-01-1990  18.190001  18.190001  18.190001  18.190001  18.190001   
2     04-01-1990  19.219999  19.219999  19.219999  19.219999  19.219999   
3     05-01-1990  20.110001  20.110001  20.110001  20.110001  20.110001   
4     08-01-1990  20.260000  20.260000  20.260000  20.260000  20.260000   
...          ...        ...        ...        ...        ...        ...   
8821  25-10-2023  19.389999  21.240000  18.860001  20.190001  20.190001   
8822  26-10-2023  21.780001  21.959999  20.219999  20.680000  20.680000   
8823  27-10-2023  20.389

/tmp/ipykernel_18396/3240749829.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_add.Date = df_add['DATE'].apply(lambda x: pd.to_datetime(x,format=date_format))
/tmp/ipykernel_18396/3240749829.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_add.Date = df_add['DATE'].apply(lambda x: pd.to_datetime(x,format=date_format))
